In [1]:
from scipy.constants import c
import numpy as np
from numpy.lib import recfunctions as rec
from astropy.io          import fits
from astropy.coordinates import SkyCoord, Angle, search_around_sky
from astropy             import units as u
from astropy import cosmology
from astropy.cosmology import FlatLambdaCDM
from matplotlib import pyplot as plt
import pandas as pd
import math
from tqdm.notebook import tqdm

In [18]:
matched_df=pd.read_pickle("matchedroutine_matches_circle.pkl")

full_stack_v=[]
population = np.empty(len(matched_df), dtype=object)
full_stack_sep=[]
full_stack_r=[]
full_stack_rich=[]
full_stack_zcen=[]

for i in range(len(matched_df)):
    full_stack_v.extend( matched_df['v'][i])
    full_stack_sep.extend( matched_df['sep2d'][i] )
    full_stack_r.extend( matched_df['x'][i] )
    population[i]=len(matched_df['v'][i])
    full_stack_rich.extend( np.repeat(matched_df['rich'][i],len(matched_df['v'][i])).tolist() )
    full_stack_zcen.extend( np.repeat(matched_df['zcen'][i],len(matched_df['v'][i])).tolist() )

wolfe=pd.DataFrame({'rich':full_stack_rich,'zcen':full_stack_zcen,'r':full_stack_r,'v':full_stack_v})

In [24]:
fname = "final_cut_catalog.npy"
dt = np.load(fname)
pax=pd.DataFrame.from_records(dt.tolist(), columns=dt.dtype.names)

In [25]:
rich=pax.LAMBDA_CHISQ
zcen=pax.BCG_SPEC_Z
r=pax.R/pax.R_LAMBDA
v=pax.Velocity

In [26]:
pax=pd.DataFrame({'rich':rich,'zcen':zcen,'r':r,'v':v});

In [39]:
r_lambda = ((wolfe.rich / 100)**0.2) #Mpc
radius=wolfe.r*r_lambda
wolfe=wolfe.loc[(radius>=0.05)]

In [40]:
len(wolfe)

255350

In [41]:
len(pax)

255350

In [62]:
wolfe_bin=round(wolfe,2)
pax_bin=round(pax,2)

In [43]:
w1=wolfe_bin.loc[ (wolfe_bin.rich>100) & (wolfe_bin.rich<190.997) ]

In [44]:
p1=pax_bin.loc[ (pax_bin.rich>100) & (pax_bin.rich<190.997) ]

In [66]:
w1=wolfe_bin
p1=pax_bin

In [67]:
merged = w1.merge(p1, how='left', indicator=True)
just_wolfe=merged.loc[ (merged._merge == 'left_only') ]

In [68]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(just_wolfe.sort_values('rich'))

Empty DataFrame
Columns: [rich, zcen, r, v, _merge]
Index: []


In [50]:
len(pax_bin.rich.unique())

4011

In [51]:
len(wolfe_bin.rich.unique())

4011

In [ ]:
#Below here is various debugging code I used to get here

In [15]:
pax_square_clust=np.load("square_cut_catalog.npy")

In [16]:
pax_square_clust.dtype.names

('MEM_MATCH_ID',
 'RA',
 'DEC',
 'MODEL_MAG',
 'MODEL_MAGERR',
 'IMAG',
 'IMAG_ERR',
 'ZRED',
 'ZRED_E',
 'ZRED_CHISQ',
 'BCG_SPEC_Z',
 'Z_SPEC_INIT',
 'Z_INIT',
 'Z',
 'LAMBDA_CHISQ',
 'LAMBDA_CHISQ_E',
 'LAMBDA_ZRED',
 'LAMBDA_ZRED_E',
 'R_LAMBDA',
 'SCALEVAL',
 'MASKFRAC',
 'C_LAMBDA',
 'C_LAMBDA_ERR',
 'MAG_LAMBDA_ERR',
 'CHISQ',
 'Z_LAMBDA',
 'Z_LAMBDA_E',
 'EBV_MEAN',
 'PHOTOID',
 'LNLAMLIKE',
 'LNBCGLIKE',
 'LNLIKE',
 'PZBINS',
 'PZ',
 'NCROSS',
 'RMASK',
 'RA_ORIG',
 'DEC_ORIG',
 'W',
 'DLAMBDA_DZ',
 'DLAMBDA_DZ2',
 'DLAMBDAVAR_DZ',
 'DLAMBDAVAR_DZ2',
 'LAMBDA_CHISQ_C',
 'LAMBDA_CHISQ_CE',
 'NCENT',
 'NCENT_GOOD',
 'RA_CENT',
 'DEC_CENT',
 'ID_CENT',
 'LAMBDA_CHISQ_CENT',
 'ZLAMBDA_CENT',
 'P_BCG',
 'P_CEN',
 'Q_CEN',
 'P_FG',
 'Q_MISS',
 'P_SAT',
 'P_C',
 'BCG_ILUM',
 'ILUM',
 'Z_LAMBDA_RAW',
 'Z_LAMBDA_E_RAW',
 'LIM_EXPTIME',
 'LIM_LIMMAG',
 'LIM_LIMMAG_HARD',
 'ZCEN_ERR',
 'Z_MEM',
 'ZMEM_ERR',
 'MEM_RA',
 'MEM_DEC')

In [17]:
np.max(pax_square_clust['MEM_MATCH_ID'])

129614

In [18]:
len(pax_square_clust)

2951649

In [19]:
matched_df=pd.read_pickle("square_matches.pkl")

In [20]:
population = np.empty(len(matched_df), dtype=object)
full_stack_sep=[]
full_stack_r=[]

for i in range(len(matched_df)):
    full_stack_sep.extend( matched_df['sep2d'][i] )
    full_stack_r.extend( matched_df['x'][i] )
    population[i]=len(matched_df['x'][i])
len(full_stack_r)

2952402

In [21]:
from astropy.coordinates import SkyCoord, Angle, search_around_sky

In [22]:
def calc_separation(ra1, dec1, ra2, dec2):
    """
    Determine the angular on sky separation between
    a member galaxy and its cluster."""
    ra_1 = Angle(ra1, unit=u.degree)
    dec_1 = Angle(dec1, unit=u.degree)
    coord1 = SkyCoord(ra=ra_1, dec=dec_1, unit = (u.deg, u.deg))

    ra_2 = Angle(ra2, unit=u.degree)
    dec_2 = Angle(dec2, unit=u.degree)
    coord2 = SkyCoord(ra=ra_2, dec=dec_2, unit = (u.deg, u.deg))

    sep = coord1.separation(coord2)
    return sep.radian

In [40]:
a1=30
d1=60
a2=50
d2=60

ang1=calc_separation(a1,d1,a2,d2)
a1=a1/360*2*math.pi
d1=d1/360*2*math.pi
a2=a2/360*2*math.pi
d2=d2/360*2*math.pi
ang2=np.arccos(np.sin(d1)*math.sin(d2)+np.cos(d1)*math.cos(d2)*np.cos(a1-a2))

In [41]:
ang2/2/math.pi*360

9.961850643857742

In [4]:
pax_vmax=np.load("pax_vmax.npy")
wolfe_vmax=np.load("wolfe_vmax.npy")

In [9]:
pax_vmax=np.unique(pax_vmax)
pax_vmax.shape

(4020,)

In [7]:
wolfe_vmax.shape

(16605,)